In [1]:
import pythonradex

# # import necessary modules
from pythonradex import radiative_transfer, helpers, LAMDA_file, atomic_transition
from scipy import constants
import numpy as np
import matplotlib.pyplot as plt

from scipy.stats import norm

In [2]:
# pyhonradex needs a file containing the atomic data, import it, this is a specific file for CO from LAMBDA database
datafilepath = r"C:\Users\alios\OneDrive - University College London\Desktop\UCL\Year3\group project\codes\co_datafile.dat"  # file downloaded from LAMDA database


# define the geometry of the nebula
geometry = "uniform sphere"

# define the parameters of the nebula
line_profile_type = "Gaussian"  # line profile, can be "Gaussian",  “rectangular”,"LVG sphere", "LVG slab”
width_v = 5 * constants.kilo  # line width in m/s, costants.kilo is 1 km/s

cloud = radiative_transfer.Cloud(
    datafilepath=datafilepath,
    geometry=geometry,
    line_profile_type=line_profile_type,
    width_v=width_v,
)

N = 1e18 / constants.centi**2  # CO column density in m-2
Tkin = 30  # kinetic temperature in [K]

# collider densities in cm-3:
para_h2_density = 1e8
ortho_h2_density = 3e8

# collider densities in m-3 (phytonradex takes the densities in m-3):
collider_densities_LTE = {
    "para-H2": para_h2_density / constants.centi**3,
    "ortho-H2": ortho_h2_density / constants.centi**3,
}

# define the background radiation field, in this case we assume that the background is zero, z is the redshift
ext_background = helpers.generate_CMB_background(z=0)

# no dust:
T_dust = 0
tau_dust = 0

# use high colliders density to simulate LTE
cloud.update_parameters(
    N=N,
    Tkin=Tkin,
    collider_densities=collider_densities_LTE,
    ext_background=ext_background,
    T_dust=T_dust,
    tau_dust=tau_dust,
)

# solve the radiative trasfer equation, i.e. calculate the level population with an iterative method
cloud.solve_radiative_transfer()

In [3]:
# store the data
data_co = LAMDA_file.read(datafilepath, read_frequencies=True)

# Initialize g_weigth
levels = data_co["levels"]
rad_transitions = data_co["radiative transitions"]
coll_transitions = data_co["collisional transitions"]
quantum_numbers = data_co["quantum numbers"]

g_weigth = np.array([level.g for level in levels])
g_up = np.array([gup.name for gup in rad_transitions])

In [4]:
number_level = np.array([level.number for level in levels])

In [5]:
number_level

array([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16,
       17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33,
       34, 35, 36, 37, 38, 39, 40])

In [6]:
rad_transitions

In [7]:
Eu = np.array([level.E for level in levels]) / constants.k

Eu_test = np.array([level.up.E for level in rad_transitions]) / constants.k

In [8]:
Eu

array([   0.        ,    5.53214517,   16.59617609,   33.19188137,
         55.31833031,   82.974736  ,  116.16031175,  154.87340764,
        199.11280441,  248.87685379,  304.16332873,  364.97043585,
        431.29580629,  503.13678297,  580.49056589,  663.35449669,
        751.72520052,  845.599158  ,  944.97313597, 1049.84303913,
       1160.20491752, 1276.0546769 , 1397.3877882 , 1524.19972692,
       1656.48539072, 1794.2399659 , 1937.45806361, 2086.1342947 ,
       2240.26298185, 2399.83801775, 2564.8534387 , 2735.3029919 ,
       2911.18013782, 3092.47804981, 3279.18990371, 3471.30844029,
       3668.82625666, 3871.73609478, 4080.02983772, 4293.69964515,
       4512.73740391])

In [9]:
Eu_test

array([   5.53214517,   16.59617609,   33.19188137,   55.31833031,
         82.974736  ,  116.16031175,  154.87340764,  199.11280441,
        248.87685379,  304.16332873,  364.97043585,  431.29580629,
        503.13678297,  580.49056589,  663.35449669,  751.72520052,
        845.599158  ,  944.97313597, 1049.84303913, 1160.20491752,
       1276.0546769 , 1397.3877882 , 1524.19972692, 1656.48539072,
       1794.2399659 , 1937.45806361, 2086.1342947 , 2240.26298185,
       2399.83801775, 2564.8534387 , 2735.3029919 , 2911.18013782,
       3092.47804981, 3279.18990371, 3471.30844029, 3668.82625666,
       3871.73609478, 4080.02983772, 4293.69964515, 4512.73740391])

In [10]:
for i, tran in enumerate(rad_transitions):
    gu = tran.up.g
    Eu = tran.up.E
    print(f"Transition {i}:")
    print("Statistical weight of upper level ", gu)
    print("Energy of upper level Eu", Eu / constants.k)
    print()

print("statsitical weight of upper level ", gu)
print("Energy of upper level Eu", Eu / constants.k)

Transition 0:
Statistical weight of upper level  3.0
Energy of upper level Eu 5.532145167854433

Transition 1:
Statistical weight of upper level  5.0
Energy of upper level Eu 16.596176090653508

Transition 2:
Statistical weight of upper level  7.0
Energy of upper level Eu 33.19188137466572

Transition 3:
Statistical weight of upper level  9.0
Energy of upper level Eu 55.31833030822088

Transition 4:
Statistical weight of upper level  11.0
Energy of upper level Eu 82.97473600266304

Transition 5:
Statistical weight of upper level  13.0
Energy of upper level Eu 116.16031175493842

Transition 6:
Statistical weight of upper level  15.0
Energy of upper level Eu 154.8734076419077

Transition 7:
Statistical weight of upper level  17.0
Energy of upper level Eu 199.11280441095295

Transition 8:
Statistical weight of upper level  19.0
Energy of upper level Eu 248.8768537863343

Transition 9:
Statistical weight of upper level  21.0
Energy of upper level Eu 304.1633287284865

Transition 10:
Statis

In [11]:
g_up[0]

np.str_('1-0')

In [12]:
def extract_upper_number(np_str):
    return int(np_str.split("-")[0])


# Example usage
first_number = extract_upper_number(g_up[0])

print(first_number)  # Output: 1

1


In [13]:
g_weigth

array([ 1.,  3.,  5.,  7.,  9., 11., 13., 15., 17., 19., 21., 23., 25.,
       27., 29., 31., 33., 35., 37., 39., 41., 43., 45., 47., 49., 51.,
       53., 55., 57., 59., 61., 63., 65., 67., 69., 71., 73., 75., 77.,
       79., 81.])

# Testing methanol 

In [14]:
# pyhonradex needs a file containing the atomic data, import it, this is a specific file for CO from LAMBDA database
datafilepath = (
    r"C:\Users\alios\OneDrive - University College London\Desktop\UCL\Year3\group project\codes\e_ch3oh.dat"  # file downloaded from LAMDA database
)


# define the geometry of the nebula
geometry = "uniform sphere"

# define the parameters of the nebula
line_profile_type = "Gaussian"  # line profile, can be "Gaussian",  “rectangular”,"LVG sphere", "LVG slab”
width_v = 5 * constants.kilo  # line width in m/s, costants.kilo is 1 km/s

cloud = radiative_transfer.Cloud(
    datafilepath=datafilepath,
    geometry=geometry,
    line_profile_type=line_profile_type,
    width_v=width_v,
)

N = 1e18 / constants.centi**2  # CO column density in m-2
Tkin = 30  # kinetic temperature in [K]

# collider densities in cm-3:
para_h2_density = 1e8
ortho_h2_density = 3e8

# collider densities in m-3 (phytonradex takes the densities in m-3):
LTE_collider_densities_h2only = {"H2": 1e8 / constants.centi**3}

# define the background radiation field, in this case we assume that the background is zero, z is the redshift
ext_background = helpers.generate_CMB_background(z=0)

# no dust:
T_dust = 0
tau_dust = 0

# use high colliders density to simulate LTE
cloud.update_parameters(
    N=N,
    Tkin=Tkin,
    collider_densities=LTE_collider_densities_h2only,
    ext_background=ext_background,
    T_dust=T_dust,
    tau_dust=tau_dust,
)

# solve the radiative trasfer equation, i.e. calculate the level population with an iterative method
cloud.solve_radiative_transfer()

79-78: tau_nu0 = -0.0215
121-120: tau_nu0 = -0.0107
137-136: tau_nu0 = -4.31e-06


c:\Users\alios\AppData\Local\Programs\Python\Python312\Lib\site-packages\pythonradex\radiative_transfer.py:110: UserWarning: some lines are overlapping, but treatement of overlapping lines not activated
  warnings.warn('some lines are overlapping, but treatement of'
c:\Users\alios\AppData\Local\Programs\Python\Python312\Lib\site-packages\pythonradex\radiative_transfer.py:333: UserWarning: negative optical depth!
  warnings.warn('negative optical depth!')


In [15]:
# store the data
data = LAMDA_file.read(datafilepath, read_frequencies=False)

# Initialize g_weigth
levels = data["levels"]
rad_transitions = data["radiative transitions"]
coll_transitions = data["collisional transitions"]
quantum_numbers = data["quantum numbers"]


In [16]:
gup = np.array([level.up.g for level in rad_transitions])
glow = np.array([level.low.g for level in rad_transitions])


In [17]:
line.low.number

NameError: name 'line' is not defined

In [ ]:
RadiativeTransition

In [ ]:
print(dir(rad_transitions))

['__add__', '__class__', '__class_getitem__', '__contains__', '__delattr__', '__delitem__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getitem__', '__getstate__', '__gt__', '__hash__', '__iadd__', '__imul__', '__init__', '__init_subclass__', '__iter__', '__le__', '__len__', '__lt__', '__mul__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__reversed__', '__rmul__', '__setattr__', '__setitem__', '__sizeof__', '__str__', '__subclasshook__', 'append', 'clear', 'copy', 'count', 'extend', 'index', 'insert', 'pop', 'remove', 'reverse', 'sort']


In [ ]:
cloud.level_pop

array([2.96536538e-02, 4.23369574e-02, 8.27875706e-03, 2.29846921e-02,
       4.69952219e-02, 3.28148662e-02, 1.76379300e-02, 3.64231762e-02,
       2.51788719e-02, 4.43434807e-02, 2.42668895e-02, 2.14388671e-02,
       2.79436121e-02, 2.69397928e-02, 3.43668936e-02, 2.38005718e-02,
       3.68177840e-02, 2.63606084e-02, 2.54170148e-02, 2.85339305e-02,
       2.24556186e-02, 2.73635326e-02, 2.18767466e-02, 2.10985333e-02,
       1.86381215e-02, 1.15020596e-02, 2.12049152e-02, 1.62471507e-02,
       1.83814869e-02, 1.08574668e-02, 1.56750509e-02, 1.38445897e-02,
       6.85533633e-03, 1.42440108e-02, 9.01570523e-03, 6.46130051e-03,
       1.09014106e-02, 1.05245606e-02, 1.12305083e-02, 9.29108293e-03,
       6.69746730e-03, 8.70130718e-03, 5.35724616e-03, 6.64812679e-03,
       6.42385084e-03, 5.66752548e-03, 6.26709408e-03, 2.80922124e-03,
       3.97289392e-03, 4.49561498e-03, 4.85356322e-03, 2.05280716e-03,
       2.33215669e-03, 3.69845984e-03, 3.57936260e-03, 2.65994033e-03,
      

In [ ]:
cloud.level_pop[4]

np.float64(0.04699522189067728)

In [ ]:
len(cloud.Tex)


2324

In [ ]:
print(dir(cloud))

['Einstein_kwargs', 'Tex', 'V_LVG_sphere', '__class__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__getstate__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'check_parameters', 'compute_residual', 'emitting_molecule', 'flux_calculator', 'fluxes_of_individual_transitions', 'geometries', 'geometry', 'geometry_name', 'get_initial_level_pop', 'level_pop', 'line_profiles', 'max_iter', 'min_iter', 'min_iter_before_ng_acceleration', 'min_tau_considered_for_convergence', 'model_grid', 'n_iter_convergence', 'ng_accelerate', 'ng_acceleration_interval', 'print_results', 'rate_equations', 'relative_convergence', 'should_be_updated', 'slow_variation_limit', 'solve_radiative_transfer', 'spectrum', 'tau_nu', 'tau_nu0_individual_transitions', 'treat_line_overl

In [ ]:
frac_pop_upper = np.array([level.pop.low.number for level in rad_transitions])

AttributeError: 'RadiativeTransition' object has no attribute 'pop'

In [18]:
rad_transitions.pop

<function list.pop(index=-1, /)>

In [19]:
print(gup[:10])

[23.  9. 23. 13. 19.  5. 29. 15. 17.  5.]


In [20]:
print(glow[:10])

[21.  7. 21. 15. 17.  7. 31. 13. 19.  7.]


In [21]:
# frac_nu = cloud.level_pop[1:]
# frac_nl = cloud.level_pop[0:-1]

In [22]:
new_Tkin = 30
frac_LTE = cloud.emitting_molecule.LTE_level_pop(T=new_Tkin)

In [23]:
frac_nu_levelpop = cloud.level_pop

In [24]:
print("4th transition upper level population fraction LTE", frac_LTE[4])
print("4th transition upper level population fraction ", frac_nu_levelpop[4])

4th transition upper level population fraction LTE 0.04678720531032945
4th transition upper level population fraction  0.04699522189067728


In [26]:
names_transitions = np.array([tran.name for tran in rad_transitions])

In [27]:
names_transitions

array(['79-78', '14-13', '96-95', ..., '227-190', '241-218', '255-248'],
      dtype='<U7')

In [28]:
names_transitions

upper_pop = cloud.level_pop[i]
lower_pop = cloud.level_pop[j]

NameError: name 'j' is not defined

In [ ]:
def get_transition_populations(names_transitions, cloud):
    """
    Relates all the transitions in names_transitions with their corresponding population densities.

    Parameters:
        names_transitions (list of str): List of transitions in the format ['79-78', '14-13', ...].
        cloud (Cloud): The Cloud object containing the level populations.

    Returns:
        tuple: Two arrays, one for upper level populations and one for lower level populations.
    """
    upper_populations = []
    lower_populations = []
    for transition in names_transitions:
        upper_level, lower_level = map(int, transition.split("-"))
        upper_pop = cloud.level_pop[upper_level]
        lower_pop = cloud.level_pop[lower_level]
        upper_populations.append(upper_pop)
        lower_populations.append(lower_pop)
    return np.array(upper_populations), np.array(lower_populations)


# Example usage
# names_transitions = ['79-78', '14-13']
upper_populations, lower_populations = get_transition_populations(names_transitions, cloud)

# Print the results
print("Upper level populations:", upper_populations)
print("Lower level populations:", lower_populations)

Upper level populations: [8.55175430e-04 3.43668936e-02 2.03570433e-04 ... 1.00378673e-15
 2.17540987e-19 6.50619327e-27]
Lower level populations: [7.74455222e-04 2.69397928e-02 1.93517320e-04 ... 6.25944849e-10
 1.68510953e-13 3.48096434e-22]


In [33]:
upper_populations[5]

np.float64(0.03281486617200964)

In [34]:
lower_populations[5]

np.float64(0.04699522189067728)

In [25]:
cloud.print_results()



  up   low      nu0 [GHz]    T_ex [K]      poplow         popup         tau_nu0
  79   78       2.937966     -17.25    0.000774455    0.000855175       -0.02147
  14   13       3.387655      20.78      0.0269398      0.0343669    2.22844e-05
  96   95       7.284957       8.67    0.000193517     0.00020357      0.0203923
  30   28       9.413483      27.93      0.0183815      0.0156751    0.000682346
  46   45       9.923130      44.65     0.00566753     0.00626709       0.231151
   5    4      12.171574      25.74      0.0469952      0.0328149        7.02246
 132  131      14.899685       6.97    2.14829e-05    1.81366e-05    0.000101716
  28   27      15.619187      38.10      0.0162472      0.0183815    0.000108306
  58   56      18.347298      30.55     0.00315441     0.00274218     7.5737e-05
   8    7      19.966178      29.28      0.0364232      0.0251789        3.63215
  42   40      22.304559      19.04     0.00669747     0.00535725    5.32485e-12
  78   77      23.383812   